# DEV - KFP

Starting to build a KFP tips notebook....

---
## Setup

### Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (Artifact Registry)
- Artifact Registry
    - to create a repository for storing custom Python packages in a GCP Project

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [3]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [4]:
try:
    import google.cloud.artifactregistry_v1
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')
    !pip install google-cloud-artifact-registry -q

### Environment

inputs:

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [6]:
REGION = 'us-central1'
EXPERIMENT = 'kfp'
SERIES = 'tips'

packages:

In [7]:
import os
import shutil

from google.cloud import aiplatform
from google.cloud import bigquery
from datetime import datetime

from google.cloud import service_usage_v1
from google.cloud import artifactregistry_v1

from datetime import datetime
from typing import NamedTuple

from kfp import dsl
from kfp.v2 import dsl as dsl2
from kfp.v2 import compiler

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  from ipykernel import kernelapp as app


clients:

In [8]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

su_client = service_usage_v1.ServiceUsageClient()
ar_client = artifactregistry_v1.ArtifactRegistryClient()

parameters:

In [9]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}/pipelines"
DIR = f"temp/{EXPERIMENT}"

In [10]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

List the service accounts current roles:

In [11]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [12]:
!rm -rf {DIR}
!mkdir -p {DIR}

### Enable APIs

Using Cloud Build and Artifact Registry requires enabling these APIs for the Google Cloud Project.

Options for enabeling these.  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Cloud Build and Enable
     - Search for Artifact Registry and Enable
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### Artifact Registry

In [13]:
artifactregistry = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
    )
).state.name


if artifactregistry == 'DISABLED':
    print(f'Artifact Registry is currently {artifactregistry} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Artifact Registry is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Artifact Registry already enabled for project: {PROJECT_ID}')

Artifact Registry already enabled for project: statmike-mlops-349915


#### Setup Artifact Registry

Artifact registry organizes artifacts with repositories.  Each repository contains packages and is designated to hold a partifcular format of package: Docker images, Python Packages and [others](https://cloud.google.com/artifact-registry/docs/supported-formats#package).

##### List Repositories

This may be empty if no repositories have been created for this project

In [14]:
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.format_.name, repo.name)

DOCKER projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915
DOCKER projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker
PYTHON projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python


#### Create/Retrieve Docker Image Repository

Create an Artifact Registry Repository to hold Docker Images created by this notebook.  First, check to see if it is already created by a previous run and retrieve it if it has.  Otherwise, create!

In [15]:
docker_repo = None
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if repo.name.endswith(PROJECT_ID):
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar_client.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{PROJECT_ID}',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {PROJECT_ID} project that holds docker images.',
                name = f'{PROJECT_ID}',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES, 'experiment': EXPERIMENT}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

Retrieved existing repo: projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915


In [16]:
print(docker_repo.format_.name, docker_repo.name)

DOCKER projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915


In [17]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"
REPOSITORY

'us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915'

In [18]:
print(f'View the repository directly in the console here:\nhttps://console.cloud.google.com/artifacts/docker/{PROJECT_ID}/{REGION}/{PROJECT_ID}?project={PROJECT_ID}')

View the repository directly in the console here:
https://console.cloud.google.com/artifacts/docker/statmike-mlops-349915/us-central1/statmike-mlops-349915?project=statmike-mlops-349915


---

## Code For Example

In [19]:
os.listdir(DIR)

[]

In [20]:
os.makedirs(DIR + '/src')

In [21]:
with open(DIR + '/src/__init__.py', 'wb') as file:
    pass

In [22]:
%%writefile {DIR}/src/train.py
# train.py
from sklearn import metrics
import sys
import my_helper

def runner(size):
    # make data
    x, y, p = my_helper.make_dataset(size)

    # fit logistic regression
    y_pred = my_helper.fit_logistic(x, y)

    # gather metrics
    cm = metrics.confusion_matrix(y, y_pred)
    auPRC = metrics.accuracy_score(y, y_pred)
    
    return cm, auPRC

cm, auPRC = runner(100)
sys.stdout.write(f'auPRC = {auPRC}')

Writing temp/kfp/src/train.py


In [23]:
%%writefile {DIR}/src/my_helper.py
# my_helper.py
import numpy as np
from sklearn import linear_model

# Make some data where y = 0, 1 for a range of x's - let y=1 be more likely as x increases
def make_dataset(size):
    x = np.random.randn(size)
    p = 1 / (1 + np.exp(-1 * (5 * x)))
    y = np.random.binomial(1, p, size)   
    return x, y, p

# fit logistic regression
def fit_logistic(x, y):
    logisticReg = linear_model.LogisticRegression()
    x2 = x.reshape(-1,1)
    fit = logisticReg.fit(x2, y)
    return fit.predict(x2)

Writing temp/kfp/src/my_helper.py


In [24]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/kfp/src/__init__.py
temp/kfp/src/train.py
temp/kfp/src/my_helper.py


---
## Local Run

In [25]:
keepdir = os.getcwd()
keepdir

'/home/jupyter/vertex-ai-mlops/Tips'

In [26]:
os.chdir(f'./{DIR}/src')

In [27]:
os.getcwd()

'/home/jupyter/vertex-ai-mlops/Tips/temp/kfp/src'

In [28]:
!python train.py

auPRC = 0.89

In [29]:
import train
#from .train import runner

auPRC = 0.94

In [30]:
train.runner(100)

(array([[47,  3],
        [ 2, 48]]),
 0.95)

In [31]:
cm, auPRC = train.runner(1000)
auPRC

0.892

In [32]:
os.chdir(keepdir)

In [33]:
os.getcwd()

'/home/jupyter/vertex-ai-mlops/Tips'

---
## Containerized Python Components

This: https://www.kubeflow.org/docs/components/pipelines/v2/author-a-pipeline/components/#2-containerized-python-components

In [34]:
# !pip install kfp==1.8.14 -q
#!pip install kfp==2.0.0b6 -q
# restart kernel

In [35]:
import kfp
kfp.__version__

'2.0.0-beta.6'

In [36]:
IMAGE = REPOSITORY + f'/{SERIES}_{EXPERIMENT}_kfp_component'
IMAGE

'us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/tips_kfp_kfp_component'

In [63]:
%%writefile {DIR}/src/my_component.py
# my_component.py
from kfp import dsl
import train

@dsl.component(
    base_image = 'python:3.7',
    target_image = 'us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/tips_kfp_kfp_component',
    packages_to_install = ['numpy', 'scikit-learn']
)
def train_model(
    size: int,
    metrics: dsl.Output[dsl.Metrics],
    class_metrics: dsl.Output[dsl.ClassificationMetrics]
):
    # run
    cm, auPRC = train.runner(size)
    
    # output
    metrics.log_metric('auPRC', auPRC)
    class_metrics.log_confusion_matrix(['Not Fraud', 'Fraud'], cm.tolist())

Overwriting temp/kfp/src/my_component.py


In [64]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/kfp/src/__init__.py
temp/kfp/src/train.py
temp/kfp/src/my_helper.py
temp/kfp/src/my_component.py
temp/kfp/src/__pycache__/my_component.cpython-37.pyc
temp/kfp/src/__pycache__/my_helper.cpython-37.pyc
temp/kfp/src/__pycache__/train.cpython-37.pyc
temp/kfp/src/__pycache__/__init__.cpython-37.pyc


In [54]:
keepdir = os.getcwd()
keepdir

'/home/jupyter/vertex-ai-mlops/Tips'

In [55]:
os.chdir(f'./{DIR}/src')

In [56]:
os.getcwd()

'/home/jupyter/vertex-ai-mlops/Tips/temp/kfp/src'

In [68]:
!kfp component build ./{DIR}/src

Building component using KFP package path: kfp==2.0.0-beta.6
No module named 'my_helper'


In [58]:
!kfp component build --help

Usage: kfp component build [OPTIONS] COMPONENTS_DIRECTORY

  Builds containers for KFP v2 Python-based components.

Options:
  --component-filepattern TEXT    Filepattern to use when searching for KFP
                                  components. The default searches all Python
                                  files in the specified directory.
  --kfp-package-path PATH         A pip-installable path to the KFP package.
  --overwrite-dockerfile          Set this to true to always generate a
                                  Dockerfile as part of the build process
  --push-image / --no-push-image  Push the built image to its remote
                                  repository.
  --help                          Show this message and exit.


In [59]:
os.chdir(keepdir)

In [61]:
os.getcwd()

'/home/jupyter/vertex-ai-mlops/Tips'

In [67]:
DIR

'temp/kfp'

---
## Custom Container - TESTING

In [75]:
IMAGE = REPOSITORY + f'/{SERIES}_{EXPERIMENT}_kfp_component'
IMAGE

'us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/tips_kfp_kfp_component'

In [45]:
%%writefile {DIR}/src/Dockerfile
FROM python:3.9
WORKDIR /my_work
RUN pip install numpy scikit-learn
## Copies the trainer code to the docker image
COPY ./* ./
## Sets up the entry point to invoke the trainer
ENTRYPOINT ["python", "-m", "train"]

Overwriting temp/kfp/src/Dockerfile


In [46]:
!docker build {DIR}/src/. -t $IMAGE

Sending build context to Docker daemon  7.168kB
Step 1/5 : FROM python:3.9
 ---> e4bf78b64f77
Step 2/5 : WORKDIR /my_work
 ---> Using cache
 ---> ea50812a783f
Step 3/5 : RUN pip install numpy scikit-learn
 ---> Using cache
 ---> 5819262c90ca
Step 4/5 : COPY ./* ./
 ---> fa83b8db3bc8
Step 5/5 : ENTRYPOINT ["python", "-m", "train"]
 ---> Running in 1e01c5ab30cf
Removing intermediate container 1e01c5ab30cf
 ---> 000af88ec09a
Successfully built 000af88ec09a
Successfully tagged us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/tips_kfp_kfp_component:latest


In [49]:
!docker run {IMAGE}

In [54]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED       STATUS       PORTS     NAMES
cfc6fa1ae606   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   8 weeks ago   Up 2 weeks             proxy-agent


In [51]:
!docker stop laughing_mirzakhani
!docker stop gifted_ardinghelli

laughing_mirzakhani
gifted_ardinghelli


In [53]:
!docker rm laughing_mirzakhani
!docker rm gifted_ardinghelli

laughing_mirzakhani
gifted_ardinghelli


---
## Pipeline (KFP) Definition

In [ ]:
@dsl.pipeline(
    name = f'series-{SERIES}-endpoint-challenger',
    description = 'Update endpoint with challenger model (conditionally).'
)
def pipeline(
    project: str,
    region: str,
    series: str,
    experiment: str,
    timestamp: str,
    bq_project: str,
    bq_dataset: str,
    bq_table: str,
    var_target: str,
    var_omit: str,
    uri: str,
    run_name: str
):
   
    # get the current model
    current_model = get_deployed_model(
        project = project,
        region = region,
        series = series
    ).set_display_name('Get Current Model').set_caching_options(False)

    # get AUC for current model
    base_model_eval = bqml_eval(
        project = project,
        region = region,
        var_target = var_target,
        bq_project = bq_project,
        bq_dataset = bq_dataset,
        bq_table = bq_table,
        bqml_model = current_model.outputs['bqml_model']
    ).set_display_name('Metric for Current Model').set_caching_options(False)
    
    # train challenger model with BQML
    challenger_model = bqml_dnn(
        project = project,
        region = region,
        series = series,
        experiment = experiment,
        timestamp = timestamp,
        var_target = var_target,
        var_omit = var_omit,
        bq_project = bq_project,
        bq_dataset = bq_dataset,
        bq_table = bq_table
    ).set_display_name('Train Challenger Model').set_caching_options(True)
    
    # get AUC for challenger model
    challenger_model_eval = bqml_eval(
        project = project,
        region = region,
        var_target = var_target,
        bq_project = bq_project,
        bq_dataset = bq_dataset,
        bq_table = bq_table,
        bqml_model = challenger_model.outputs['bqml_model']
    ).set_display_name('Metric for Challenger Model').set_caching_options(False)
    challenger_model_eval.after(challenger_model)
    
    # compare models
    compare = model_compare(
        base_metric = base_model_eval.outputs["metric"],
        challenger_metric = challenger_model_eval.outputs["metric"]
    ).set_display_name('Compare Models')
    
    # conditional deployment
    with dsl.Condition(
        compare.output == 'true',
        name = "replace_model"
    ):
        # export BQML model to Vertex AI Model Registry
        export = bqml_export(
            project = project,
            region = region,
            series = series,
            experiment = experiment,
            timestamp = timestamp,
            uri = uri,
            run_name = run_name,
            bq_project = bq_project,
            bq_dataset = bq_dataset,
            bqml_model = challenger_model.outputs["bqml_model"]
        ).set_display_name('Export BQML Model')
        
        # replace model on endpoint (03b)
        replace = endpoint_update(
            project = project,
            region = region,
            series = series,
            experiment = experiment,
            vertex_endpoint = current_model.outputs['vertex_endpoint'],
            vertex_model = export.outputs['vertex_model']
        ).set_display_name('Deploy The Challenger Model')

---
## Compile And Run Pipeline

### Compile Inputs

In [ ]:
parameter_values = {
    "project" : PROJECT_ID,
    "region" : REGION,
    "series": SERIES,
    "experiment" : EXPERIMENT,
    "timestamp": TIMESTAMP,
    "bq_project": BQ_PROJECT,
    "bq_dataset": BQ_DATASET,
    "bq_table": BQ_TABLE,
    "var_target": VAR_TARGET,
    "var_omit": VAR_OMIT,
    "uri": URI,
    "run_name": RUN_NAME
}

### Compile Pipeline

In [ ]:
# from kfp.v2 import compiler
kfp.v2.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path = f"{DIR}/{EXPERIMENT}.json"
)

### Define Pipeline Job

Using compiled pipeline:

In [ ]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f'{EXPERIMENT}',
    template_path = f"{DIR}/{EXPERIMENT}.json",
    pipeline_root = f"{URI}/pipeline_root",
    parameter_values = parameter_values,
    enable_caching = False, # overrides all component/task settings
    labels = {'series': SERIES, 'experiment': EXPERIMENT}
)

### Submit Pipeline Job

In [ ]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

Using the following link to view the job in the GCP console:

In [ ]:
print(f'The Dashboard can be viewed here:\n{pipeline_job._dashboard_uri()}')

#### Wait On Pipeline Job

In [ ]:
pipeline_job.wait()

### Retrieve Pipeline Information

In [ ]:
aiplatform.get_pipeline_df(pipeline = f'series-{SERIES}-endpoint-challenger')

## Review Pipeline Run

<p aligh="center"><center><img src="../architectures/notebooks/03/pipeline_ex2.png" width="75%"></center></p>

---
## Remove Resources
see notebook "99 - Cleanup"